# Read Dataset CoNLL2003

In [1]:
file_name = 'data/conll2003/ner/train.txt'
# file_name2 = 'data/conll2003/ner/train.txt'

In [2]:
from dataloaders.ner_conll2003 import get_labels, load_data_from_file

/media/SSD/NDT34-venv/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
data = []
sentence = []
label = []

In [4]:
f = open(file_name, "r")
# f2 = open(file_name2, "w")
# f2.write("-DOCSTART- -X- -X- O\n\n")

for line in f:
    if len(line)==0 or line.startswith('-DOCSTART') or line[0] == "\n":
        if len(sentence) > 0:
            data.append((sentence, label))
            sentence = []
            label = []
            # f2.write("\n")
        continue
    splits = line.split(' ')
    word = splits[0]
    tag = splits[-1][:-1]
    
    sentence.append(word)
    label.append(tag)
    # f2.write("{} {}\n".format(word, tag))

if len(sentence) > 0:
    data.append((sentence, label))
    sentence = []
    label = []

f.close()
# f2.close()

In [5]:
len(data)

14041

In [6]:
# l = []
# for (sentences, labels) in data:
#     l.extend(labels)

In [7]:
# set(l)

# Create noisy dataset

In [8]:
percent = 0.3
number_change = int(len(data)*percent)
indexes_change = []

In [9]:
import random
import numpy as np

while len(indexes_change) < number_change:
    ind = random.choice(np.arange(0, len(data)))
    if ind not in indexes_change:
        indexes_change.append(ind)

In [10]:
len(indexes_change) # index của các câu sẽ bị đảo nhãn

4212

In [11]:
len(set(indexes_change))

4212

In [12]:
target_file = 'data/conll2003/ner/noise_BItags_30sen_30tok.txt'
f = open(target_file, "w")
f.write("-DOCSTART- -X- -X- O\n\n")

for index, (words, tags) in enumerate(data):
    # make lists copy
    words = words.copy()
    tags = tags.copy()
    change = [0] * len(words)
    origin_tags = tags.copy()
    
    if index in indexes_change:
        # change some tag in tags
        # select some ind to change
        num_change_tags = int(len(tags) * percent)
        ind_change_tags = []
        while len(ind_change_tags) < num_change_tags:
            ind = random.choice(np.arange(0, len(tags)))
            if ind not in ind_change_tags:
                ind_change_tags.append(ind)
        
        # Change
        for i in ind_change_tags:
            old_tag = new_tag = tags[i]
            while old_tag == new_tag:
                new_tag = random.choice(get_labels())
            tags[i] = new_tag
            change[i] = 1
        
        # Recheck
        for i, (w, c, t, o) in enumerate(zip(words, change, tags, origin_tags)):
            # print("{:10}\t\t{:3}\t{:5}\t{:5}".format(w,c,t,o))
            if c==1 and t[:1] == 'I':
                if i-1>=0 and tags[i-1] != 'O':
                    tags[i] = 'I' + tags[i-1][1:]
                elif (i-1>=0 and tags[i-1] == 'O') or i == 0:
                    tags[i] = 'B' + tags[i][1:]
    #     # For test
    #     print()
    #     for i, (w, c, t, o) in enumerate(zip(words, change, tags, origin_tags)):
    #         print("{:10}\t\t{:3}\t{:5}\t{:5}".format(w,c,t,o))
    #     print('-'*50)
    #     c+=1
    # if c == 5:
    #     break
            
        
    # write to file
    for i in range(len(words)):
        f.write("{} {} {} {}\n".format(words[i], origin_tags[i], change[i], tags[i]))
    f.write("\n")
f.close()